# ECS

In [1]:
import os

## Clusters
- Siempre que se corren Tasks o Services se schedulean para correr en un Cluster.
- Se pueden mezclar muchos tipos de Instancias, las cuales pueden abarcar multiples AZ.
- Las Instancias solo pueden pertenecer a un Cluster en un momento dado.

In [20]:
# Creando un cluster llamado flask-app
!aws ecs create-cluster --cluster-name fsilvestre-flask-app --profile dh


{
    "cluster": {
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "clusterName": "flask-app",
        "status": "ACTIVE",
        "registeredContainerInstancesCount": 0,
        "runningTasksCount": 0,
        "pendingTasksCount": 0,
        "activeServicesCount": 0,
        "statistics": [],
        "tags": [],
        "settings": [
            {
                "name": "containerInsights",
                "value": "disabled"
            }
        ],
        "capacityProviders": [],
        "defaultCapacityProviderStrategy": []
    }
}


In [3]:
# Listando todos los clusters
!aws ecs list-clusters --profile dh

{
    "clusterArns": [
        "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app-ccosta", 
        "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app-leo-f", 
        "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app", 
        "arn:aws:ecs:us-east-1:622305974757:cluster/fsilvestre-flask-app", 
        "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app-mbustamante", 
        "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app-dabadie-std", 
        "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app-efilgueira"
    ]
}


In [22]:
# Detalle del cluster flask-app
!aws ecs describe-clusters --clusters fsilvestre-flask-app --profile dh

{
    "clusters": [
        {
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "clusterName": "flask-app",
            "status": "ACTIVE",
            "registeredContainerInstancesCount": 0,
            "runningTasksCount": 0,
            "pendingTasksCount": 0,
            "activeServicesCount": 0,
            "statistics": [],
            "tags": [],
            "settings": [
                {
                    "name": "containerInsights",
                    "value": "disabled"
                }
            ],
            "capacityProviders": [],
            "defaultCapacityProviderStrategy": []
        }
    ],
    "failures": []
}


In [6]:
# Borrar el cluster
!aws ecs delete-cluster --cluster fsilvestre-flask-app --profile dh

{
    "cluster": {
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "clusterName": "flask-app",
        "status": "INACTIVE",
        "registeredContainerInstancesCount": 0,
        "runningTasksCount": 0,
        "pendingTasksCount": 0,
        "activeServicesCount": 0,
        "statistics": [],
        "tags": [],
        "settings": [
            {
                "name": "containerInsights",
                "value": "disabled"
            }
        ],
        "capacityProviders": [],
        "defaultCapacityProviderStrategy": []
    }
}


## Container Agent
- Permite que una Instancia EC2 se una a un Cluster.
- Ya viene en algunas AMIs o se puede instalar en cualquier Instancia.
- Es Open Source, escrito en Go y se puede ver su codigo [acá](https://github.com/aws/amazon-ecs-agent)
- Tambien hay una imagen de [Docker](https://hub.docker.com/r/amazon/amazon-ecs-agent/)
- Tiene muchas [variables de configuracion](https://docs.aws.amazon.com/AmazonECS/latest/developerguide/ecs-agent-config.html), aunque con las que vienen por default suele ser suficiente para arrancar
- Reporta el estado de sus Tasks al Scheduler.

In [10]:
# Creamos un Bucket para guardar la configuracion
!aws s3 mb s3://fsilvestre-flask-app  --profile dh

make_bucket: flask-app-dh


In [5]:
# El archivo de configuracion que debe tener cualquier instancia que queremos que se registre en nuestro Cluster
!cat container-agent/ecs.config

ECS_CLUSTER=fsilvestre-flask-app


In [4]:
# Lo subimos al Bucket
!aws s3 cp container-agent/ecs.config s3://fsilvestre-flask-app --profile dh

upload: container-agent/ecs.config to s3://fsilvestre-flask-app/ecs.config


## Container Instances
- Instancia EC2 registrada en un Cluster.
- Se conecta via el Container Agent.
- Estados en el Ciclo de vida.
- `ACTIVE` and `connected`: El Container Agent registró la Instancia en el Cluster y está lista para correr tareas.
- `ACTIVE` and `disconnected`: Cuando **detenemos** la instancia, por lo tanto las tareas que estaban corriendo en ella se van a detener. Si lo iniciamos de vuelta se volveria a conectar con el Container Agent.
- `INACTIVE`: Cuando la instancia se **termina** o se retira del Cluster.

In [6]:
!aws ec2 run-instances \
--count 1 \
--profile dh \
--instance-type t2.micro \
--image-id ami-0b22c910bce7178b6 \
--iam-instance-profile Name=ecsInstanceRoleFlask \
--key-name fsilvestre-digital \
--security-group-ids sg-934830e5 \
--user-data file://container-agent/copy-ecs-config-to-s3.sh \
--tag-specifications 'ResourceType=instance,Tags=[{Key=Course,Value=cloud_architect}, {Key=Name,Value=fsilvestre-flask-app}]' 'ResourceType=volume,Tags=[{Key=Course,Value=cloud_architect}]'

{
    "Instances": [
        {
            "Monitoring": {
                "State": "disabled"
            }, 
            "PublicDnsName": "", 
            "StateReason": {
                "Message": "pending", 
                "Code": "pending"
            }, 
            "State": {
                "Code": 0, 
                "Name": "pending"
            }, 
            "EbsOptimized": false, 
            "LaunchTime": "2020-06-23T13:58:54.000Z", 
            "PrivateIpAddress": "172.31.62.133", 
            "ProductCodes": [], 
            "VpcId": "vpc-8c5cdff7", 
            "CpuOptions": {
                "CoreCount": 1, 
                "ThreadsPerCore": 1
            }, 
            "StateTransitionReason": "", 
            "InstanceId": "i-03ff0dfae5300df5b", 
            "ImageId": "ami-0b22c910bce7178b6", 
            "PrivateDnsName": "ip-172-31-62-133.ec2.internal", 
            "KeyName": "fsilvestre-digital", 
            "SecurityGroups": [
                {
          

- La [AMI](https://docs.aws.amazon.com/AmazonECS/latest/developerguide/ecs-optimized_AMI.html) que estamos lanzando. Especificamente [esta](https://us-east-1.console.aws.amazon.com/systems-manager/parameters/aws/service/ecs/optimized-ami/amazon-linux-2/recommended/image_id/description?region=us-east-1#)
- El [rol](https://console.aws.amazon.com/iam/home?region=us-east-1#/roles/ecsInstanceRoleFlask) que estamos usando
- El security group que estamos usando: [default](https://console.aws.amazon.com/ec2/v2/home?region=us-east-1#SecurityGroups:group-id=sg-934830e5)
- Le estamos pasando este script de arranque a nuestra instancia:

In [7]:
# Instalar el aws-cli y bajar la configuracion de ECS.
!cat container-agent/copy-ecs-config-to-s3.sh

#!/bin/bash
yum install -y aws-cli
aws s3 cp s3://fsilvestre-flask-app/ecs.config /etc/ecs/ecs.config


In [9]:
# ¿Cuantas instancias tenemos en el cluster?
!aws ecs list-container-instances --cluster fsilvestre-flask-app --profile dh

{
    "containerInstanceArns": [
        "arn:aws:ecs:us-east-1:622305974757:container-instance/98fff291-37b9-4731-a325-9accd1d560ba"
    ]
}


In [10]:
# Detalle de una de las instancias de arriba
!aws ecs describe-container-instances --cluster fsilvestre-flask-app --container-instances arn:aws:ecs:us-east-1:622305974757:container-instance/98fff291-37b9-4731-a325-9accd1d560ba --profile dh

{
    "failures": [], 
    "containerInstances": [
        {
            "status": "ACTIVE", 
            "registeredAt": 1592920789.371, 
            "registeredResources": [
                {
                    "integerValue": 1024, 
                    "longValue": 0, 
                    "type": "INTEGER", 
                    "name": "CPU", 
                    "doubleValue": 0.0
                }, 
                {
                    "integerValue": 983, 
                    "longValue": 0, 
                    "type": "INTEGER", 
                    "name": "MEMORY", 
                    "doubleValue": 0.0
                }, 
                {
                    "name": "PORTS", 
                    "longValue": 0, 
                    "doubleValue": 0.0, 
                    "stringSetValue": [
                        "22", 
                        "2376", 
                        "2375", 
                        "51678", 
                        "51679"
                   

## Task Definitions
- Describe como una imagen Docker debe **correr**.
- Pueden tenes uno o mas containers.
- Posiblemente con volumenes.
- Una aplicación puede estar compuesta por una o mas definiciones de tareas.
- Los containers que están en la misma Task Definition corren en la **misma instancia**.
- Ejemplos:
  - Frontend/Backend que deben estar detras de un nginx como proxy reverso.
  - Cuando queremos usar un cache local.
- Los Services se crean a partir de las Task Definitions.
- Constan de:
  - Una familia, es un nombre que no tiene nada que ver con el nombre del docker.
  - Una definicion del container, similar al YAML de Docker Compose.
- Se incrementa en uno la version cada vez que la modificamos.

In [31]:
# Nuestra primera Task Definition, solo un container
# 50 MB de RAM, si se pasa de eso, la matamos
# 102 maximo no mas de 10% de CPU
# family el nombre de lo que va a hacer
!cat task-definition/web-task-definition.json

{
  "containerDefinitions": [
    {
      "name": "nginx",
      "image": "nginx",
      "portMappings": [
        {
          "containerPort": 80,
          "hostPort": 80
        }
      ],
      "memory": 50,
      "cpu": 102
    }
  ],
  "family": "web"
}


¿Me ayudan a escribir  un `docker run` equivalente?
> docker run -t nginx -p 80:80 nginx

In [32]:
# Registremos la Task Definition
!aws ecs register-task-definition --cli-input-json file://task-definition/web-task-definition.json  --profile dh

{
    "taskDefinition": {
        "taskDefinitionArn": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
        "containerDefinitions": [
            {
                "name": "nginx",
                "image": "nginx",
                "cpu": 102,
                "memory": 50,
                "portMappings": [
                    {
                        "containerPort": 80,
                        "hostPort": 80,
                        "protocol": "tcp"
                    }
                ],
                "essential": true,
                "environment": [],
                "mountPoints": [],
                "volumesFrom": []
            }
        ],
        "family": "web",
        "revision": 26,
        "volumes": [],
        "status": "ACTIVE",
        "placementConstraints": [],
        "compatibilities": [
            "EC2"
        ]
    }
}


Fijense que no colocamos el cluster ¿Se imaginan por que?

In [1]:
# Las viejas no se pueden borrar, quedan inactivas
!aws ecs list-task-definition-families  --profile dh

{
    "families": [
        "Pipeline-bleiva-DeploymentPipeline-KSA80D1WD0AH-ecs-cluster-BlueService-9JVNP9KJGN3T-TaskDefinition-1I1F0M3J8A9G2", 
        "Pipeline-bleiva-DeploymentPipeline-KSA80D1WD0AH-ecs-cluster-BlueService-9JVNP9KJGN3T-TaskDefinition-7CJB8A1R0O8O", 
        "Pipeline-bleiva-DeploymentPipeline-KSA80D1WD0AH-ecs-cluster-GreenService-K34WAEG1WEJZ-TaskDefinition-1I2BQRHGVA3LO", 
        "Pipeline-bleiva-DeploymentPipeline-KSA80D1WD0AH-ecs-cluster-GreenService-K34WAEG1WEJZ-TaskDefinition-1P661VL8GZCIM", 
        "Pipeline-bleiva-DeploymentPipeline-KSA80D1WD0AH-ecs-cluster-GreenService-K34WAEG1WEJZ-TaskDefinition-AWSBU2Z5F6EZ", 
        "PipelineBleiva-DeploymentPipeline-47HJ1QO7VV0D-ecs-cluster-BlueService-K0DWCVH7DCRS-TaskDefinition-E7UAJ3HPK3CO", 
        "PipelineBleiva-DeploymentPipeline-47HJ1QO7VV0D-ecs-cluster-GreenService-1G887SARZWWJB-TaskDefinition-1UOI21ABNGBDX", 
        "PipelineBleiva-DeploymentPipeline-47HJ1QO7VV0D-ecs-cluster-GreenService-1G887SARZWWJB-Tas

In [2]:
# ¿Que task definitions activas tenemos hasta ahora?
!aws ecs list-task-definitions  --profile dh

{
    "taskDefinitionArns": [
        "arn:aws:ecs:us-east-1:622305974757:task-definition/web:33", 
        "arn:aws:ecs:us-east-1:622305974757:task-definition/web:34", 
        "arn:aws:ecs:us-east-1:622305974757:task-definition/web:35"
    ]
}


In [3]:
# Detalle completo de una task definition
!aws ecs describe-task-definition --task-definition web:35  --profile dh

{
    "taskDefinition": {
        "status": "ACTIVE", 
        "family": "web", 
        "placementConstraints": [], 
        "compatibilities": [
            "EC2"
        ], 
        "volumes": [], 
        "taskDefinitionArn": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:35", 
        "containerDefinitions": [
            {
                "environment": [], 
                "name": "nginx", 
                "mountPoints": [], 
                "image": "nginx", 
                "cpu": 102, 
                "portMappings": [
                    {
                        "protocol": "tcp", 
                        "containerPort": 80, 
                        "hostPort": 80
                    }
                ], 
                "memory": 50, 
                "essential": true, 
                "volumesFrom": []
            }
        ], 
        "revision": 35
    }
}


In [38]:
# Registremos una nueva version de la Task Definition
!aws ecs register-task-definition --cli-input-json file://task-definition/web-task-definition.json  --profile dh

{
    "taskDefinition": {
        "taskDefinitionArn": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:27",
        "containerDefinitions": [
            {
                "name": "nginx",
                "image": "nginx",
                "cpu": 102,
                "memory": 50,
                "portMappings": [
                    {
                        "containerPort": 80,
                        "hostPort": 80,
                        "protocol": "tcp"
                    }
                ],
                "essential": true,
                "environment": [],
                "mountPoints": [],
                "volumesFrom": []
            }
        ],
        "family": "web",
        "revision": 27,
        "volumes": [],
        "status": "ACTIVE",
        "placementConstraints": [],
        "compatibilities": [
            "EC2"
        ]
    }
}


In [4]:
# Vemos la nueva version registrada
!aws ecs list-task-definitions  --profile dh

{
    "taskDefinitionArns": [
        "arn:aws:ecs:us-east-1:622305974757:task-definition/web:33", 
        "arn:aws:ecs:us-east-1:622305974757:task-definition/web:34", 
        "arn:aws:ecs:us-east-1:622305974757:task-definition/web:35"
    ]
}


In [7]:
# Eliminamos la ultima versión
!aws ecs deregister-task-definition --task-definition web:34  --profile dh

{
    "taskDefinition": {
        "status": "INACTIVE", 
        "family": "web", 
        "placementConstraints": [], 
        "compatibilities": [
            "EC2"
        ], 
        "volumes": [], 
        "taskDefinitionArn": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:34", 
        "containerDefinitions": [
            {
                "environment": [], 
                "name": "nginx", 
                "mountPoints": [], 
                "image": "nginx", 
                "cpu": 102, 
                "portMappings": [
                    {
                        "protocol": "tcp", 
                        "containerPort": 80, 
                        "hostPort": 80
                    }
                ], 
                "memory": 50, 
                "essential": true, 
                "volumesFrom": []
            }
        ], 
        "revision": 34
    }
}


In [8]:
# Ya no la vemos mas, pero quedó como `INACTIVE`
!aws ecs list-task-definitions  --profile dh

{
    "taskDefinitionArns": [
        "arn:aws:ecs:us-east-1:622305974757:task-definition/web:33"
    ]
}


In [9]:
# En caso de dudas, consulte su `help`
!aws ecs register-task-definition help  --profile dh

REGISTER-TASK-DEFINITION()                          REGISTER-TASK-DEFINITION()



NNAAMMEE
       register-task-definition -

DDEESSCCRRIIPPTTIIOONN
       Registers  a  new task definition from the supplied ffaammiillyy and ccoonnttaaiinn--
       eerrDDeeffiinniittiioonnss . Optionally, you can add data volumes to your containers
       with  the vvoolluummeess parameter. For more information about task definition
       parameters and defaults, see _A_m_a_z_o_n _E_C_S _T_a_s_k _D_e_f_i_n_i_t_i_o_n_s in the  _A_m_a_z_o_n
       _E_l_a_s_t_i_c _C_o_n_t_a_i_n_e_r _S_e_r_v_i_c_e _D_e_v_e_l_o_p_e_r _G_u_i_d_e .

       You  can specify an IAM role for your task with the ttaasskkRRoolleeAArrnn parame-
       ter. When you specify an IAM role for a task, its containers  can  then
       use  the latest versions of the AWS CLI or SDKs to make API requests to
       

In [44]:
# ¿Que atributos tenia una task definition?
!aws ecs register-task-definition --generate-cli-skeleton  --profile dh

{
    "family": "",
    "taskRoleArn": "",
    "executionRoleArn": "",
    "networkMode": "none",
    "containerDefinitions": [
        {
            "name": "",
            "image": "",
            "repositoryCredentials": {
                "credentialsParameter": ""
            },
            "cpu": 0,
            "memory": 0,
            "memoryReservation": 0,
            "links": [
                ""
            ],
            "portMappings": [
                {
                    "containerPort": 0,
                    "hostPort": 0,
                    "protocol": "tcp"
                }
            ],
            "essential": true,
            "entryPoint": [
                ""
            ],
            "command": [
                ""
            ],
            "environment": [
                {
                    "name": "",
                    "value": ""
                }
            ],
            "environmentFiles": [
                {
                    "value": "",
 

## Scheduler
- Se encarga de aprovechar los recursos de computo del Cluster de la mejor manera posible.
- No deberia importarnos que instancia corre un Service o Task especifica.
- 3 maneras de schedulear tareas en un Cluster
  - Services
  - Running Tasks
  - Starting Tasks

## Services
- Services son procesos que **siempre estan corriendo** y **no tienen estado**. Por ejemplo servidores web.
- Cuando se registra un Service se le indica cuantas intancias (ejemplares) de la Task Defininion deben estar corriendo.
- Si alguna de ellas cae, el Scheduler **crea una nueva**.
- Se puede enganchar el Service a un ELB. 
- 3 pasos para colocar un Service en el Cluster
  - Comparar la definición de la tarea con el estado del Cluster.
  - Con la lista de instancias en el cluster, se fija cuantas estan corriendo el servicio hasta ahora en esta AZ, si la cantidad es menor crea la cantidad restante en distintas AZs.
  - Con la lista de instancias en el cluster, se fija cuantas estan corriendo el servicio hasta ahora en las instancias, si la cantidad es menor crea la cantidad restante en distintas instancias, aunque todas entren en una.

In [45]:
# Scheduleamos un servicio a partir de la ultima version de la Task Definition `web`
!aws ecs create-service --cluster fsilvestre-flask-app --task-definition web --service-name web-service --desired-count 1  --profile dh

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 1,
        "runningCount": 0,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/0320419872898933672",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
                "desiredCount": 1,
                "pendingCount": 0,
                "runningCount": 0,
                "createdAt": "2020-06-18T16:3

In [46]:
# Veamos si se instanció el servicio
!aws ecs list-services --cluster flask-app

{
    "serviceArns": [
        "arn:aws:ecs:us-east-1:622305974757:service/web-service"
    ]
}


In [47]:
# Inspeccionamos el detalle del servicio
# Podemos ver `events`
!aws ecs describe-services --cluster flask-app --services web-service

{
    "services": [
        {
            "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
            "serviceName": "web-service",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "loadBalancers": [],
            "serviceRegistries": [],
            "status": "ACTIVE",
            "desiredCount": 1,
            "runningCount": 1,
            "pendingCount": 0,
            "launchType": "EC2",
            "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
            "deploymentConfiguration": {
                "maximumPercent": 200,
                "minimumHealthyPercent": 100
            },
            "deployments": [
                {
                    "id": "ecs-svc/0320419872898933672",
                    "status": "PRIMARY",
                    "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
                    "desiredCount": 1,
                    

In [56]:
# !aws ec2 describe-instances

In [98]:
# Veamos si funciona el servicio en su DNS publico
# Buscar el DNS publico de la instancia
!aws ec2 describe-instances | jq '.Reservations[1].Instances[0].PublicDnsName'

""


In [62]:
# ¿Duplicamos la capacidad?
# "desiredCount": 2,
# "runningCount": 1, 
!aws ecs update-service --cluster flask-app --task-definition web --service web-service --desired-count 2

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 3,
        "runningCount": 2,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/0320419872898933672",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
                "desiredCount": 3,
                "pendingCount": 0,
                "runningCount": 2,
                "createdAt": "2020-06-18T16:3

In [74]:
# ¿Funcionó?
!aws ecs describe-services --cluster flask-app --services web-service

{
    "services": [
        {
            "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
            "serviceName": "web-service",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "loadBalancers": [],
            "serviceRegistries": [],
            "status": "ACTIVE",
            "desiredCount": 3,
            "runningCount": 3,
            "pendingCount": 0,
            "launchType": "EC2",
            "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
            "deploymentConfiguration": {
                "maximumPercent": 200,
                "minimumHealthyPercent": 100
            },
            "deployments": [
                {
                    "id": "ecs-svc/0320419872898933672",
                    "status": "PRIMARY",
                    "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
                    "desiredCount": 3,
                    

¿Que pasó?

In [64]:
# Lanzamos otra instancia
!aws ec2 run-instances \
--count 1 \
--instance-type t2.micro \
--image-id ami-0b22c910bce7178b6 \
--iam-instance-profile Name=ecsInstanceRoleFlask \
--key-name dh-north-virginia \
--security-group-ids sg-934830e5 \
--user-data file://container-agent/copy-ecs-config-to-s3.sh \
--tag-specifications 'ResourceType=instance,Tags=[{Key=Course,Value=cloud_architect}]' 'ResourceType=volume,Tags=[{Key=Course,Value=cloud_architect}]'

{
    "Groups": [],
    "Instances": [
        {
            "AmiLaunchIndex": 0,
            "ImageId": "ami-0b22c910bce7178b6",
            "InstanceId": "i-07bff7be0af09c8fc",
            "InstanceType": "t2.micro",
            "KeyName": "dh-north-virginia",
            "LaunchTime": "2020-06-18T19:53:37+00:00",
            "Monitoring": {
                "State": "disabled"
            },
            "Placement": {
                "AvailabilityZone": "us-east-1e",
                "GroupName": "",
                "Tenancy": "default"
            },
            "PrivateDnsName": "ip-172-31-56-208.ec2.internal",
            "PrivateIpAddress": "172.31.56.208",
            "ProductCodes": [],
            "PublicDnsName": "",
            "State": {
                "Code": 0,
                "Name": "pending"
            },
            "StateTransitionReason": "",
            "SubnetId": "subnet-e14466de",
            "VpcId": "vpc-8c5cdff7",
            "Architecture": "x86_64",
      

In [76]:
# Matemos todos los servicios para poder borrarlo
!aws ecs update-service --cluster flask-app --task-definition web --service web-service --desired-count 0

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 0,
        "runningCount": 2,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/0320419872898933672",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:26",
                "desiredCount": 0,
                "pendingCount": 0,
                "runningCount": 2,
                "createdAt": "2020-06-18T16:3

In [1]:
# Borremos el servicio
# "status": "DRAINING",
!aws ecs delete-service --cluster fsilvestre-flask-app --service web-service --profile dh


An error occurred (InvalidParameterException) when calling the DeleteService operation: The service cannot be stopped while it is scaled above 0.


In [78]:
# No quedaron mas servicios
!aws ecs list-services --cluster flask-app

{
    "serviceArns": []
}


In [79]:
# ¿Y como es el skeleton de un servicio?
!aws ecs create-service --generate-cli-skeleton

{
    "cluster": "",
    "serviceName": "",
    "taskDefinition": "",
    "loadBalancers": [
        {
            "targetGroupArn": "",
            "loadBalancerName": "",
            "containerName": "",
            "containerPort": 0
        }
    ],
    "serviceRegistries": [
        {
            "registryArn": "",
            "port": 0,
            "containerName": "",
            "containerPort": 0
        }
    ],
    "desiredCount": 0,
    "clientToken": "",
    "launchType": "FARGATE",
    "capacityProviderStrategy": [
        {
            "capacityProvider": "",
            "weight": 0,
            "base": 0
        }
    ],
    "platformVersion": "",
    "role": "",
    "deploymentConfiguration": {
        "maximumPercent": 0,
        "minimumHealthyPercent": 0
    },
    "placementConstraints": [
        {
            "type": "memberOf",
            "expression": ""
        }
    ],
    "placementStrategy": [
        {
            "type": "spread",
            "field": ""

## Running Tasks
- Tareas de **corta vida**, tareas unicas o aisladas, corren y terminan.
- Por ejemplo obtener miniaturas de una foto, encoding de video o migraciones de bases de datos.
- `run-task` lo ubica en el cluster de forma **aleatoria** minimizando la sobrecarga sobre una sola instancia.

In [80]:
# Arranquemos nginx como una task (no es lo ideal)
# "lastStatus": "PENDING",
!aws ecs run-task --cluster flask-app --task-definition web --count 1

{
    "tasks": [
        {
            "attachments": [],
            "availabilityZone": "us-east-1c",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "containerInstanceArn": "arn:aws:ecs:us-east-1:622305974757:container-instance/1993f7db-fde2-4d90-b6b0-3b19a9b08b34",
            "containers": [
                {
                    "containerArn": "arn:aws:ecs:us-east-1:622305974757:container/34d1b09d-f104-4c66-8b3c-c08de8d28f70",
                    "taskArn": "arn:aws:ecs:us-east-1:622305974757:task/c187a422-3f3f-4b21-b53a-77e9c1d63904",
                    "name": "nginx",
                    "image": "nginx",
                    "lastStatus": "PENDING",
                    "networkInterfaces": [],
                    "cpu": "102",
                    "memory": "50"
                }
            ],
            "cpu": "102",
            "createdAt": "2020-06-18T17:42:36.751000-03:00",
            "desiredStatus": "RUNNING",
            

In [81]:
# Veamos cuantas tasks tenemos corriendo en el cluster
!aws ecs list-tasks --cluster flask-app

{
    "taskArns": [
        "arn:aws:ecs:us-east-1:622305974757:task/c187a422-3f3f-4b21-b53a-77e9c1d63904"
    ]
}


In [82]:
# Detengamos la tarea anterior
# "lastStatus": "RUNNING",
# "desiredStatus": "STOPPED",
!aws ecs stop-task --cluster flask-app --task arn:aws:ecs:us-east-1:622305974757:task/c187a422-3f3f-4b21-b53a-77e9c1d63904

{
    "task": {
        "attachments": [],
        "availabilityZone": "us-east-1c",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "connectivity": "CONNECTED",
        "connectivityAt": "2020-06-18T17:42:36.751000-03:00",
        "containerInstanceArn": "arn:aws:ecs:us-east-1:622305974757:container-instance/1993f7db-fde2-4d90-b6b0-3b19a9b08b34",
        "containers": [
            {
                "containerArn": "arn:aws:ecs:us-east-1:622305974757:container/34d1b09d-f104-4c66-8b3c-c08de8d28f70",
                "taskArn": "arn:aws:ecs:us-east-1:622305974757:task/c187a422-3f3f-4b21-b53a-77e9c1d63904",
                "name": "nginx",
                "image": "nginx",
                "runtimeId": "b3cda8d88d56f2501af41280e0cb22d0a21ccd70adf1702cfb259db4ec117f61",
                "lastStatus": "RUNNING",
                "networkBindings": [
                    {
                        "bindIP": "0.0.0.0",
                        "containerPort": 

In [83]:
# ¿Se detuvo la tarea anterior?
!aws ecs list-tasks --cluster flask-app

{
    "taskArns": []
}


## Starting Tasks
- Muy parecida a `run-task`.
- Con `start-task` se puede elegir en que instancia va a correr en el Cluster.
- Para casos en los que en el Cluster tenemos instancias con mejores recursos de CPU (Video Transcoding) o GPU (Training).
- Hay mas Schedulers dando vueltas por el mercado.

In [84]:
# ¿Que instancias tenemos?
!aws ecs list-container-instances --cluster flask-app

{
    "containerInstanceArns": [
        "arn:aws:ecs:us-east-1:622305974757:container-instance/18f34b11-ed75-42e6-be3a-9941ba54526f",
        "arn:aws:ecs:us-east-1:622305974757:container-instance/1993f7db-fde2-4d90-b6b0-3b19a9b08b34",
        "arn:aws:ecs:us-east-1:622305974757:container-instance/df487d2d-dad0-44b0-8c8e-13bbe1a1725b"
    ]
}


In [85]:
# Con el ARN de arriba arrancamos la tarea
!aws ecs start-task --cluster flask-app --task-definition web --container-instances arn:aws:ecs:us-east-1:622305974757:container-instance/18f34b11-ed75-42e6-be3a-9941ba54526f

{
    "tasks": [
        {
            "attachments": [],
            "availabilityZone": "us-east-1e",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "containerInstanceArn": "arn:aws:ecs:us-east-1:622305974757:container-instance/18f34b11-ed75-42e6-be3a-9941ba54526f",
            "containers": [
                {
                    "containerArn": "arn:aws:ecs:us-east-1:622305974757:container/6541bf69-3870-48ec-8096-53437767e033",
                    "taskArn": "arn:aws:ecs:us-east-1:622305974757:task/9a99e543-fd18-40c5-9689-105fe9f70e0e",
                    "name": "nginx",
                    "image": "nginx",
                    "lastStatus": "PENDING",
                    "networkInterfaces": [],
                    "cpu": "102",
                    "memory": "50"
                }
            ],
            "cpu": "102",
            "createdAt": "2020-06-18T17:45:51.721000-03:00",
            "desiredStatus": "RUNNING",
            

In [86]:
# ¿Arrancó?
!aws ecs list-tasks --cluster flask-app

{
    "taskArns": [
        "arn:aws:ecs:us-east-1:622305974757:task/9a99e543-fd18-40c5-9689-105fe9f70e0e"
    ]
}


In [87]:
# Con el taskArn de la ejecución anterior
# "lastStatus": "RUNNING",
# "desiredStatus": "STOPPED",
!aws ecs stop-task --cluster flask-app --task arn:aws:ecs:us-east-1:622305974757:task/9a99e543-fd18-40c5-9689-105fe9f70e0e

{
    "task": {
        "attachments": [],
        "availabilityZone": "us-east-1e",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "connectivity": "CONNECTED",
        "connectivityAt": "2020-06-18T17:45:51.721000-03:00",
        "containerInstanceArn": "arn:aws:ecs:us-east-1:622305974757:container-instance/18f34b11-ed75-42e6-be3a-9941ba54526f",
        "containers": [
            {
                "containerArn": "arn:aws:ecs:us-east-1:622305974757:container/6541bf69-3870-48ec-8096-53437767e033",
                "taskArn": "arn:aws:ecs:us-east-1:622305974757:task/9a99e543-fd18-40c5-9689-105fe9f70e0e",
                "name": "nginx",
                "image": "nginx",
                "runtimeId": "429d3c8e0725d6a0fca0c0990f0d911cc3a294b808635a3d67995333121e69c7",
                "lastStatus": "RUNNING",
                "networkBindings": [
                    {
                        "bindIP": "0.0.0.0",
                        "containerPort": 

In [88]:
# ¿Se detuvo la tarea anterior?
!aws ecs list-tasks --cluster flask-app

{
    "taskArns": []
}


## Container propio

Deployemos un container de nuestra autoria: https://hub.docker.com/r/rsavoie/flask

In [89]:
# ¿Que tenia?
!cat flask-app/Dockerfile

# Imagen padre desde la que vamos a heredar. Podriamos usar otra?
FROM python:3.6

# Directorio adentro del container en donde vamos a trabajar
# Es como hacer un cd, sino estariamos trabajando en el /
WORKDIR /app

# Contexto: Copia todo el contenido de este directerio adentro del container en la carpeta /app
COPY . /app

# Correr un comando, en este caso para instalar las dependencias en requirements.txt
RUN pip install -r requirements.txt

# Abre el puerto 5000 del container
EXPOSE 5000

# Cuando el container se lance va a ejecutar este comando
CMD ["python", "app.py"]

In [90]:
# ¿Funciona?
!docker run -d --rm --name flask -p 80:5000 rsavoie/flask

Unable to find image 'rsavoie/flask:latest' locally
latest: Pulling from rsavoie/flask

d16361e0: Pulling fs layer 
28769df1: Pulling fs layer 
949d0793: Pulling fs layer 
92510b70: Pulling fs layer 
ecf88e6a: Pulling fs layer 
c1cbf347: Pulling fs layer 
6b55cd0f: Pulling fs layer 
a2441bbb: Pulling fs layer 
28117005: Pulling fs layer 
f2e458d8: Pulling fs layer 
587c1675: Pulling fs layer 
Digest: sha256:6549033da69c9a0da8a9096424d595e9a658d55fae5f3fe467ff3ea6726a645d
Status: Downloaded newer image for rsavoie/flask:latest
061d7d0d2f9fa58018c6bd7479cc2a735d9a6cd7cdcc04096954fdf18332cde3


In [91]:
!curl http://localhost

{
  "message": "Hello! I'm inside a container"
}


In [93]:
# Cambiamos la Task Definition anterior por una actualizada para usar con flask en lugar del nginx, seguimos en el puerto 80
!cat task-definition/flask-app-task-definition.json

{
  "containerDefinitions": [
    {
      "name": "flask",
      "image": "rsavoie/flask",
      "portMappings": [
        {
          "containerPort": 5000,
          "hostPort": 80
        }
      ],
      "memory": 50,
      "cpu": 102
    }
  ],
  "family": "web"
}


In [94]:
# La registramos en el Cluster
!aws ecs register-task-definition --cli-input-json file://task-definition/flask-app-task-definition.json

{
    "taskDefinition": {
        "taskDefinitionArn": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
        "containerDefinitions": [
            {
                "name": "flask",
                "image": "rsavoie/flask",
                "cpu": 102,
                "memory": 50,
                "portMappings": [
                    {
                        "containerPort": 5000,
                        "hostPort": 80,
                        "protocol": "tcp"
                    }
                ],
                "essential": true,
                "environment": [],
                "mountPoints": [],
                "volumesFrom": []
            }
        ],
        "family": "web",
        "revision": 28,
        "volumes": [],
        "status": "ACTIVE",
        "placementConstraints": [],
        "compatibilities": [
            "EC2"
        ]
    }
}


In [95]:
# Scheduleamos un servicio a partir de la nueva Task Definition
!aws ecs create-service --cluster flask-app --service-name web-service --task-definition web --desired-count 1

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 1,
        "runningCount": 0,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/3837189649463570480",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
                "desiredCount": 1,
                "pendingCount": 0,
                "runningCount": 0,
                "createdAt": "2020-06-18T17:5

In [96]:
# Vemos si se creo el servicio
!aws ecs list-services --cluster flask-app

{
    "serviceArns": [
        "arn:aws:ecs:us-east-1:622305974757:service/web-service"
    ]
}


In [97]:
# Detalle del servicio anterior
!aws ecs describe-services --cluster flask-app --services arn:aws:ecs:us-east-1:622305974757:service/web-service

{
    "services": [
        {
            "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
            "serviceName": "web-service",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "loadBalancers": [],
            "serviceRegistries": [],
            "status": "ACTIVE",
            "desiredCount": 1,
            "runningCount": 1,
            "pendingCount": 0,
            "launchType": "EC2",
            "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
            "deploymentConfiguration": {
                "maximumPercent": 200,
                "minimumHealthyPercent": 100
            },
            "deployments": [
                {
                    "id": "ecs-svc/3837189649463570480",
                    "status": "PRIMARY",
                    "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
                    "desiredCount": 1,
                    

In [99]:
# Veamos si funciona el servicio en su DNS publico
# Buscar el DNS publico de la instancia
!aws ec2 describe-instances | jq '.Reservations[1].Instances[0].PublicDnsName'

""


In [100]:
# Dupliquemos capacidad
!aws ecs update-service --cluster flask-app --service web-service --task-definition web --desired-count 2

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 2,
        "runningCount": 1,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/3837189649463570480",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
                "desiredCount": 2,
                "pendingCount": 0,
                "runningCount": 1,
                "createdAt": "2020-06-18T17:5

In [101]:
!aws ecs list-services --cluster flask-app

{
    "serviceArns": [
        "arn:aws:ecs:us-east-1:622305974757:service/web-service"
    ]
}


In [102]:
!aws ecs describe-services --cluster flask-app --services arn:aws:ecs:us-east-1:622305974757:service/web-service

{
    "services": [
        {
            "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
            "serviceName": "web-service",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "loadBalancers": [],
            "serviceRegistries": [],
            "status": "ACTIVE",
            "desiredCount": 2,
            "runningCount": 2,
            "pendingCount": 0,
            "launchType": "EC2",
            "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
            "deploymentConfiguration": {
                "maximumPercent": 200,
                "minimumHealthyPercent": 100
            },
            "deployments": [
                {
                    "id": "ecs-svc/3837189649463570480",
                    "status": "PRIMARY",
                    "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
                    "desiredCount": 2,
                    

In [150]:
# !aws ec2 describe-instances | jq '.Reservations[0].Instances[0].PublicDnsName'
# !aws ec2 describe-instances | jq '.Reservations[1].Instances[0].PublicDnsName'

## Actualizando el container propio

In [107]:
# Leamos una variable de entorno
f"Hello! I'm inside a container in {os.environ.get('ENVIRONMENT')}"

"Hello! I'm inside a container in None"

In [113]:
# El contexto de construccion es la carpeta `flask-app`
!docker build -t flask flask-app

Sending build context to Docker daemon  6.144kB
Step 1/6 : FROM python:3.6
 ---> 13efce2de907
Step 2/6 : WORKDIR /app
 ---> Using cache
 ---> 9f6727b6e674
Step 3/6 : COPY . /app
 ---> 195bab49a671
Step 4/6 : RUN pip install -r requirements.txt
 ---> Running in 86f59614bfb3
Removing intermediate container 86f59614bfb3
 ---> cc519290433f
Step 5/6 : EXPOSE 5000
 ---> Running in ee527b8f4cc5
Removing intermediate container ee527b8f4cc5
 ---> c4b17e2659c2
Step 6/6 : CMD ["python", "app.py"]
 ---> Running in c4b8ff2f7341
Removing intermediate container c4b8ff2f7341
 ---> 35505e0c9185
Successfully built 35505e0c9185
Successfully tagged flask:latest


In [115]:
# ¿Funciona?
# !docker rm -f flask
!docker run -d --rm --name flask -p 80:5000 -e ENVIRONMENT=dev flask

flask
18b40d87f32076703d54d76d03fca24d92bad593d91b0d100ca7f16b69ce9147


In [116]:
!curl http://localhost

{
  "message": "Hello! I'm inside a container in dev"
}


In [117]:
# Taggeamos
!docker tag flask rsavoie/flask

In [118]:
!docker images

REPOSITORY            TAG                 IMAGE ID            CREATED             SIZE
flask                 latest              35505e0c9185        29 seconds ago      924MB
rsavoie/flask         latest              35505e0c9185        29 seconds ago      924MB
<none>                <none>              57abca8d1391        2 minutes ago       924MB
flask-redis           latest              b6f1abe0b6c3        3 hours ago         924MB
rsavoie/flask-redis   latest              b6f1abe0b6c3        3 hours ago         924MB
flask-redis_web       latest              2580bf21145f        4 hours ago         924MB
ubuntu                latest              74435f89ab78        44 hours ago        73.9MB
redis                 alpine              b546e82a6d0e        8 days ago          31.5MB
python                3.6                 13efce2de907        9 days ago          914MB
rsavoie/flask         <none>              1d60bf4072d4        7 months ago        923MB


In [119]:
# Pusheamos
!docker push rsavoie/flask

The push refers to repository [docker.io/rsavoie/flask]

6e9e3a1e: Preparing 
6d4fb492: Preparing 
9c495a27: Preparing 
703d6103: Preparing 
f6aa0d01: Preparing 
b9f75f56: Preparing 
305e9fd3: Preparing 
18a2e1b1: Preparing 
4ca91984: Preparing 
de4639e0: Preparing 
89d2d3c5: Preparing 
e9e3a1e: Pushed   10.21MB/9.509MBsk-redis [12Alatest: digest: sha256:e1e56c777a6e82aac7b5b751c2667abe66159d781fe946a086005bc707fd2db6 size: 2842


https://hub.docker.com/r/rsavoie/flask

Nuestros services siguen con la version anterior

In [120]:
# Matamos las dos instancias previas del servicio
!aws ecs update-service --cluster flask-app --service web-service --task-definition web --desired-count 0

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 0,
        "runningCount": 0,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/3837189649463570480",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
                "desiredCount": 0,
                "pendingCount": 0,
                "runningCount": 0,
                "createdAt": "2020-06-18T17:5

In [121]:
!aws ecs describe-services --cluster flask-app --services web-service

{
    "services": [
        {
            "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
            "serviceName": "web-service",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "loadBalancers": [],
            "serviceRegistries": [],
            "status": "ACTIVE",
            "desiredCount": 0,
            "runningCount": 0,
            "pendingCount": 0,
            "launchType": "EC2",
            "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
            "deploymentConfiguration": {
                "maximumPercent": 200,
                "minimumHealthyPercent": 100
            },
            "deployments": [
                {
                    "id": "ecs-svc/3837189649463570480",
                    "status": "PRIMARY",
                    "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
                    "desiredCount": 0,
                    

In [122]:
!aws ecs update-service --cluster flask-app --service web-service --task-definition web --desired-count 2

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 2,
        "runningCount": 0,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/3837189649463570480",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:28",
                "desiredCount": 2,
                "pendingCount": 0,
                "runningCount": 0,
                "createdAt": "2020-06-18T17:5

## ¿Y las variables de entorno?

Hay muchas formas de pasarle variables de entorno a nuestros containers  
https://dev.to/sagarjauhari/3-ways-to-handle-secrets-in-aws-ecs-tasks-2olb

In [123]:
!cat task-definition/flask-app-env-task-definition.json

{
  "containerDefinitions": [
    {
      "name": "flask",
      "image": "rsavoie/flask",
      "portMappings": [
        {
          "containerPort": 5000,
          "hostPort": 80
        }
      ],
      "environment" : [
        { "name" : "ENVIRONMENT", "value" : "production" }
      ],      
      "memory": 50,
      "cpu": 102
    }
  ],
  "family": "web"
}


In [124]:
!aws ecs register-task-definition --cli-input-json file://task-definition/flask-app-env-task-definition.json

{
    "taskDefinition": {
        "taskDefinitionArn": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:29",
        "containerDefinitions": [
            {
                "name": "flask",
                "image": "rsavoie/flask",
                "cpu": 102,
                "memory": 50,
                "portMappings": [
                    {
                        "containerPort": 5000,
                        "hostPort": 80,
                        "protocol": "tcp"
                    }
                ],
                "essential": true,
                "environment": [
                    {
                        "name": "ENVIRONMENT",
                        "value": "production"
                    }
                ],
                "mountPoints": [],
                "volumesFrom": []
            }
        ],
        "family": "web",
        "revision": 29,
        "volumes": [],
        "status": "ACTIVE",
        "placementConstraints": [],
        "compatibilitie

In [126]:
# Bajamos a cero el conteo de tasks
!aws ecs update-service --cluster flask-app --service web-service --task-definition web --desired-count 0

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 0,
        "runningCount": 0,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:29",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/4255718551320865842",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:29",
                "desiredCount": 0,
                "pendingCount": 0,
                "runningCount": 0,
                "createdAt": "2020-06-18T18:2

In [127]:
!aws ecs describe-services --cluster flask-app --services web-service

{
    "services": [
        {
            "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
            "serviceName": "web-service",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "loadBalancers": [],
            "serviceRegistries": [],
            "status": "ACTIVE",
            "desiredCount": 0,
            "runningCount": 0,
            "pendingCount": 0,
            "launchType": "EC2",
            "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:29",
            "deploymentConfiguration": {
                "maximumPercent": 200,
                "minimumHealthyPercent": 100
            },
            "deployments": [
                {
                    "id": "ecs-svc/4255718551320865842",
                    "status": "PRIMARY",
                    "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:29",
                    "desiredCount": 0,
                    

In [128]:
# Para que las nuevas tasks se levanten con la nueva definicion con la variable de entorno
!aws ecs update-service --cluster flask-app --service web-service --task-definition web --desired-count 2

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 2,
        "runningCount": 0,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:29",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/4255718551320865842",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:29",
                "desiredCount": 2,
                "pendingCount": 0,
                "runningCount": 0,
                "createdAt": "2020-06-18T18:2

## Servicios compuestos de dos containers

In [129]:
# El Docker Compose...
!cat flask-redis/docker-compose.yml

version: '3'
services:
  web:
    build: .
    ports:
      - "80:5000"
  redis:
    image: "redis:alpine"

In [131]:
# ...y la Task Definition son muy parecidos
!cat task-definition/flask-app-redis-task-definition.json

{
  "containerDefinitions": [
    {
      "name": "flask-redis",
      "image": "rsavoie/flask-redis",
      "portMappings": [
        {
          "containerPort": 5000,
          "hostPort": 80
        }
      ],
      "environment" : [
        { "name" : "ENVIRONMENT", "value" : "production" }
      ],      
      "memory": 50,
      "cpu": 102,
      "links": [
        "redis"
      ]      
    },
    {
      "name": "redis",
      "image": "redis:alpine",
      "portMappings": [
        {
          "containerPort": 6379,
          "hostPort": 6379
        }
      ],    
      "memory": 150,
      "cpu": 102
    }
  ],
  "family": "web"
}


In [132]:
!aws ecs register-task-definition --family web --cli-input-json file://task-definition/flask-app-redis-task-definition.json

{
    "taskDefinition": {
        "taskDefinitionArn": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:30",
        "containerDefinitions": [
            {
                "name": "flask-redis",
                "image": "rsavoie/flask-redis",
                "cpu": 102,
                "memory": 50,
                "links": [
                    "redis"
                ],
                "portMappings": [
                    {
                        "containerPort": 5000,
                        "hostPort": 80,
                        "protocol": "tcp"
                    }
                ],
                "essential": true,
                "environment": [
                    {
                        "name": "ENVIRONMENT",
                        "value": "production"
                    }
                ],
                "mountPoints": [],
                "volumesFrom": []
            },
            {
                "name": "redis",
                "image": "redis:alpi

In [133]:
# Bajamos a cero el conteo de tasks
!aws ecs update-service --cluster flask-app --task-definition web --service arn:aws:ecs:us-east-1:622305974757:service/web-service --desired-count 0

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 0,
        "runningCount": 2,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:30",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/5967705781542921733",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:30",
                "desiredCount": 0,
                "pendingCount": 0,
                "runningCount": 0,
                "createdAt": "2020-06-18T18:2

In [134]:
!aws ecs describe-services --cluster flask-app --services arn:aws:ecs:us-east-1:622305974757:service/web-service

{
    "services": [
        {
            "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
            "serviceName": "web-service",
            "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
            "loadBalancers": [],
            "serviceRegistries": [],
            "status": "ACTIVE",
            "desiredCount": 0,
            "runningCount": 0,
            "pendingCount": 0,
            "launchType": "EC2",
            "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:30",
            "deploymentConfiguration": {
                "maximumPercent": 200,
                "minimumHealthyPercent": 100
            },
            "deployments": [
                {
                    "id": "ecs-svc/5967705781542921733",
                    "status": "PRIMARY",
                    "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:30",
                    "desiredCount": 0,
                    

In [135]:
# Para que las nuevas tasks se levanten con la nueva definicion con la variable de entorno
!aws ecs update-service --cluster flask-app --task-definition web --service arn:aws:ecs:us-east-1:622305974757:service/web-service --desired-count 2

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:622305974757:service/web-service",
        "serviceName": "web-service",
        "clusterArn": "arn:aws:ecs:us-east-1:622305974757:cluster/flask-app",
        "loadBalancers": [],
        "serviceRegistries": [],
        "status": "ACTIVE",
        "desiredCount": 2,
        "runningCount": 0,
        "pendingCount": 0,
        "launchType": "EC2",
        "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:30",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/5967705781542921733",
                "status": "PRIMARY",
                "taskDefinition": "arn:aws:ecs:us-east-1:622305974757:task-definition/web:30",
                "desiredCount": 2,
                "pendingCount": 0,
                "runningCount": 0,
                "createdAt": "2020-06-18T18:2

## ECR
- Hasta ahora venimos usando todas imagenes de Docker **públicas**.
- La imagen que veniamos trabajando hasta ahora en Docker Hub vamos a trasladarla a un **Repositorio Privado**.
- Elastic Container Registry es un Registro de Docker gestionado por AWS
- Toda cuenta de AWS ya tiene seteada un Registro de Docker
- En nuestro caso es http://622305974757.dkr.ecr.us-east-1.amazonaws.com/
- El 622305974757 es el AWS Account ID, el mismo en el que loguean https://622305974757.signin.aws.amazon.com/console
- Ahi podemos agregar repositorios, todos los que necesitemos.
- Un repositorio deberia ser un microservicio: Un modulo de Frontend o Backend por ejemplo
- Podria ser el equivalente a un repositorio de GitHub
- Se integra con IAM, para que solo ciertos usuarios o instancias accedan a ciertos repositorios.
- Se paga por lo que usa en
  - Almacenaniento: Peso de las imagenes
  - Ancho de banda: Subir o bajar al registro
- Construyan imagenes optimas ;-) Amiguense con https://hub.docker.com/_/alpine

In [137]:
# Obtenemos un token para que nuestro Docker CLI pueda pushear a nuestro Docker Registry en ECR
!aws ecr get-login-password

eyJwYXlsb2FkIjoiWGNPaUZrSGlCNDlSVjBsZDlCNm5LbjhadENkcC9iN0VObE1NblNyNHl3dmFGSXlYU0I4RWpnRXBHbHNnTjE0NVdUcnFaSkZzZ3g1UmVpMStJOTFVemtFcGRCZy9nSGdMYlIzKzNsMnNGc2hPQUVFZ21JazdHOW5ITGw1Q0dWSlppZFo1bDdBOFVNeTBaRmxUN01DYmdNTnRFMW9YQUZVb0pscURxcXJvTEI3M2lFNEZIYm9SckhJVzliMzVxVnJESWZDR3VmVFRQR0lGcEtEbDNUTmgzU0tDM1RoK2x4aHpiTlNUcm5rd2Y5WkxCZzhBNVJZeitZQ2s2MTJoU0kyekJ4TDN3Z1B3NmF2VXFVcnhhY3B4MDA5VnVIeWx6Mm82a3UxcGxrVklNeFl5TkNnSEJWTDdCcHF3R2tFTER3Q3N2elF5Z3p3T04xV202eldpcStmKzdlbTdpSjhiSDhQMjFGZU10VlRzWjZBWGoyY3JiQ3hYM0RlelRMcW9Xc3Nmc3RjY3NzeHYyQ2dvdXB0UXJLZThpTUgxYVFHOHN1V2lwaEtUVGJ6Yk50ZDd0T2Rtb3lROGswNWo0Wjh4WW5GZ2V0WTkrYkxiK3Bpb3l0d1RDdTlrK2ZZZ2pFYkxWUUIwVHhsT2JiS2xVaHRwYng4d08vM29hQ1VyaTl1WjhsRlJGV00raTdid3hTeHpzZ1FlZkROU3Z1TnNPNmhyakw3Wm9HcG1FQUUzaDdiVnlWZTdEaFdNd0NwZ2Vac1daa0Y3M3dmTnRUMWUvWWd6SE5MSE80QnZKUWtzODMrL2pXV21Bb3VHU1hEOTRvOXAyMVl6QVBQQnF4UDBPOHV3Uk5zdU1iUFZ5NkwwcFlTSXR6RkxRRk5PbDdHTHJVbXFkK0o5bjNudER3SGJxV2hBcSt2dVd3WkpUM3dsT2ZzbU1KU0xTeDdHMFpGVTdqYnMwOW11aytIOEk5bFU3NlI3aU01TXJZ

In [138]:
# Autenticamos en docker, el -e fue deprecado
!docker login -u AWS -p eyJwYXlsb2FkIjoiWGNPaUZrSGlCNDlSVjBsZDlCNm5LbjhadENkcC9iN0VObE1NblNyNHl3dmFGSXlYU0I4RWpnRXBHbHNnTjE0NVdUcnFaSkZzZ3g1UmVpMStJOTFVemtFcGRCZy9nSGdMYlIzKzNsMnNGc2hPQUVFZ21JazdHOW5ITGw1Q0dWSlppZFo1bDdBOFVNeTBaRmxUN01DYmdNTnRFMW9YQUZVb0pscURxcXJvTEI3M2lFNEZIYm9SckhJVzliMzVxVnJESWZDR3VmVFRQR0lGcEtEbDNUTmgzU0tDM1RoK2x4aHpiTlNUcm5rd2Y5WkxCZzhBNVJZeitZQ2s2MTJoU0kyekJ4TDN3Z1B3NmF2VXFVcnhhY3B4MDA5VnVIeWx6Mm82a3UxcGxrVklNeFl5TkNnSEJWTDdCcHF3R2tFTER3Q3N2elF5Z3p3T04xV202eldpcStmKzdlbTdpSjhiSDhQMjFGZU10VlRzWjZBWGoyY3JiQ3hYM0RlelRMcW9Xc3Nmc3RjY3NzeHYyQ2dvdXB0UXJLZThpTUgxYVFHOHN1V2lwaEtUVGJ6Yk50ZDd0T2Rtb3lROGswNWo0Wjh4WW5GZ2V0WTkrYkxiK3Bpb3l0d1RDdTlrK2ZZZ2pFYkxWUUIwVHhsT2JiS2xVaHRwYng4d08vM29hQ1VyaTl1WjhsRlJGV00raTdid3hTeHpzZ1FlZkROU3Z1TnNPNmhyakw3Wm9HcG1FQUUzaDdiVnlWZTdEaFdNd0NwZ2Vac1daa0Y3M3dmTnRUMWUvWWd6SE5MSE80QnZKUWtzODMrL2pXV21Bb3VHU1hEOTRvOXAyMVl6QVBQQnF4UDBPOHV3Uk5zdU1iUFZ5NkwwcFlTSXR6RkxRRk5PbDdHTHJVbXFkK0o5bjNudER3SGJxV2hBcSt2dVd3WkpUM3dsT2ZzbU1KU0xTeDdHMFpGVTdqYnMwOW11aytIOEk5bFU3NlI3aU01TXJZS3JRVjhiWEJXR0YxM0pkT2htcFpmM0NxcjVDN2haeUtnT2IyMjEvOVRBM3o1UUN6bVVrZVZFVno5L052cTMrUk9GL25hc1Nrc1dLZWxKS1hvSmhiUXdJbXo1eUhsM3hkM2p2Zk5VczkzNGUza21DTjlFbU5vZmJHRzJPUFFnT0tJd2NwSUdQRFh0S3dyWmhhUDdpWm1UWndHYS92MUJjOHBRQmNqeVpZdTdVODRFd04vSVAzWDcyc0gwZ21iVHN2NThmYXBwRlo0SFd6UXVTcThhYkJnWTJLTy84TTVuQkhRS0w5UmNJWjV4MTJsL3luQ1c2QjNJVVVFMHpjL2FOczBIY3JrMkY0OU4yWGc3MWpQV2VjQmZNRjZLQzFJT2wwdkdSa0pXTGl6bTFvL3RobkRsZzJqL0k1TDliUDQvMkJ5M2NVOW5PblAzMHQ5VjF0SXFqT21jNC9kV3FmNzZKNUxkR0t2anBka3I5T21rUE0yOFlsM3dMcGo4UFJGcWY0aGlvK0VSR1FIa1FFcURZb0J1NjVCdnNlQ3lVcnNlVGZQaXB0Z1VRaEJPYUFYOVBYby9ONmkwNlY1cElRPT0iLCJkYXRha2V5IjoiQVFFQkFIaHdtMFlhSVNKZVJ0Sm01bjFHNnVxZWVrWHVvWFhQZTVVRmNlOVJxOC8xNHdBQUFINHdmQVlKS29aSWh2Y05BUWNHb0c4d2JRSUJBREJvQmdrcWhraUc5dzBCQndFd0hnWUpZSVpJQVdVREJBRXVNQkVFREV0SWlmNEY1S1lrdmFBRWl3SUJFSUE3M0lRN1pOUXdNN3c4M0ZMc3hCY1Nyc1RVbVRUSG91T0JQOTE4TmxOclRoNkxpMS9LbTJ5OVBiaVEvbEhDdk9Camx5MWJ1aFdISmlRVnZSUT0iLCJ2ZXJzaW9uIjoiMiIsInR5cGUiOiJEQVRBX0tFWSIsImV4cGlyYXRpb24iOjE1OTI1NTkzNzV9 https://622305974757.dkr.ecr.us-east-1.amazonaws.com

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ramiro/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [139]:
# Mismo nombre de repositorio en ECR y en local
!aws ecr create-repository --repository-name rsavoie/flask-redis

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:622305974757:repository/rsavoie/flask-redis",
        "registryId": "622305974757",
        "repositoryName": "rsavoie/flask-redis",
        "repositoryUri": "622305974757.dkr.ecr.us-east-1.amazonaws.com/rsavoie/flask-redis",
        "createdAt": "2020-06-18T18:38:07-03:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        }
    }
}


In [140]:
# ¿Se creo el repo?
!aws ecr describe-repositories

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:622305974757:repository/rsavoie/flask-redis",
            "registryId": "622305974757",
            "repositoryName": "rsavoie/flask-redis",
            "repositoryUri": "622305974757.dkr.ecr.us-east-1.amazonaws.com/rsavoie/flask-redis",
            "createdAt": "2020-06-18T18:38:07-03:00",
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            }
        }
    ]
}


In [141]:
# No deberiamos tener ninguna imagen
!aws ecr list-images --repository-name rsavoie/flask-redis

{
    "imageIds": []
}


In [142]:
# Taggeamos la imagen para el nuevo repo de ECR
!docker tag flask-redis 622305974757.dkr.ecr.us-east-1.amazonaws.com/rsavoie/flask-redis

In [144]:
# La misma imagen tageada tres veces
!docker images

REPOSITORY                                                         TAG                 IMAGE ID            CREATED             SIZE
flask                                                              latest              35505e0c9185        18 minutes ago      924MB
rsavoie/flask                                                      latest              35505e0c9185        18 minutes ago      924MB
<none>                                                             <none>              57abca8d1391        20 minutes ago      924MB
622305974757.dkr.ecr.us-east-1.amazonaws.com/rsavoie/flask-redis   latest              b6f1abe0b6c3        4 hours ago         924MB
flask-redis                                                        latest              b6f1abe0b6c3        4 hours ago         924MB
rsavoie/flask-redis                                                latest              b6f1abe0b6c3        4 hours ago         924MB
flask-redis_web                                                    lat

In [147]:
# Lo pusheamos
!docker push 622305974757.dkr.ecr.us-east-1.amazonaws.com/rsavoie/flask-redis

The push refers to repository [622305974757.dkr.ecr.us-east-1.amazonaws.com/rsavoie/flask-redis]

bc9c7c19: Preparing 
94e60cab: Preparing 
9c495a27: Preparing 
703d6103: Preparing 
f6aa0d01: Preparing 
b9f75f56: Preparing 
305e9fd3: Preparing 
18a2e1b1: Preparing 
4ca91984: Preparing 
de4639e0: Preparing 
89d2d3c5: Preparing 
4ca91984: Pushing  57.16MB/145.5MBPushing  31.11MB/509.9MB^C


In [149]:
# Ahora la imagen de flask-redis se lee desde nuestro Docker Registry privado
!cat task-definition/flask-app-redis-ecr-task-definition.json

{
  "containerDefinitions": [
    {
      "name": "flask-redis",
      "image": "622305974757.dkr.ecr.us-east-1.amazonaws.com/rsavoie/flask-redis",
      "portMappings": [
        {
          "containerPort": 5000,
          "hostPort": 80
        }
      ],
      "environment" : [
        { "name" : "ENVIRONMENT", "value" : "production" }
      ],      
      "memory": 50,
      "cpu": 102,
      "links": [
        "redis"
      ]      
    },
    {
      "name": "redis",
      "image": "redis:alpine",
      "portMappings": [
        {
          "containerPort": 6379,
          "hostPort": 6379
        }
      ],    
      "memory": 150,
      "cpu": 102
    }
  ],
  "family": "web"
}


In [ ]:
# Actualizamos la Task Definition
!aws ecs register-task-definition --family web --cli-input-json file://task-definition/flask-app-redis-ecr-task-definition.json

In [ ]:
!aws ecs update-service --cluster flask-app --task-definition web --service arn:aws:ecs:us-east-1:622305974757:service/web-service --desired-count 0

In [ ]:
# Nueva version del servicio
!aws ecs describe-services --cluster flask-app --services arn:aws:ecs:us-east-1:622305974757:service/web-service

## Eliminando el cluster

In [ ]:
# Bajamos el conteo de instancias a cero
!aws ecs update-service --cluster flask-app --task-definition web --service arn:aws:ecs:us-east-1:622305974757:service/web-service --desired-count 0

In [ ]:
# Instance ID de la maquina 0
# !aws ec2 describe-instances | jq '.Reservations[0].Instances[0].InstanceId'

# Instance ID de la maquina 1
# !aws ec2 describe-instances | jq '.Reservations[1].Instances[0].InstanceId'

# Con los IDs de arriba
# !aws ec2 terminate-instances --instance-ids i-00115b208fae69fc3 i-08fafd1a82c5cdb00

In [ ]:
# Vemos que versiones tenemos registradas de la Task
!aws ecs list-task-definitions

In [ ]:
# Eliminamos todas las versiones
!aws ecs deregister-task-definition --task-definition web:25

In [ ]:
# Eliminamos el repo, primero las imagenes y despues el repo
!aws ecr delete-repository --repository-name rsavoie/flask-redis --force

In [ ]:
!aws ecs list-services --cluster flask-app

In [ ]:
# Borramos el servicio web-service
!aws ecs delete-service --cluster flask-app --service web-service

In [ ]:
# Borramos el cluster
!aws ecs delete-cluster --cluster flask-app